<a href="https://colab.research.google.com/github/morenourey/DSC_NUCLIO/blob/main/TFM_DATA_PREPOCESSING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")

In [3]:
# fechas
from datetime import datetime

# gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

# algoritmos de clasificación
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# algoritmos de regresión
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# métricas
import sklearn.metrics as metrics

# hiperparametrización
from sklearn.model_selection import GridSearchCV

# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

import xgboost as xgb

from numpy import random
random.seed(42)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df_product = pd.read_csv('/content/drive/MyDrive/NUCLIO/TFM_FINTEC/products_df.csv')

df_sociodemo = pd.read_csv('/content/drive/MyDrive/NUCLIO/TFM_FINTEC/sociodemographic_df.csv')

df_comercial = pd.read_csv('/content/drive/MyDrive/NUCLIO/TFM_FINTEC/commercial_activity_df.csv')

In [11]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 18 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   pk_cid              int64  
 2   pk_partition        object 
 3   short_term_deposit  int64  
 4   loans               int64  
 5   mortgage            int64  
 6   funds               int64  
 7   securities          int64  
 8   long_term_deposit   int64  
 9   em_account_pp       int64  
 10  credit_card         int64  
 11  payroll             float64
 12  pension_plan        float64
 13  payroll_account     int64  
 14  emc_account         int64  
 15  debit_card          int64  
 16  em_account_p        int64  
 17  em_acount           int64  
dtypes: float64(2), int64(15), object(1)
memory usage: 818.9+ MB


In [15]:
df_product.nunique()

Unnamed: 0            5962924
pk_cid                 456373
pk_partition               17
short_term_deposit          2
loans                       2
mortgage                    2
funds                       2
securities                  2
long_term_deposit           2
em_account_pp               1
credit_card                 2
payroll                     2
pension_plan                2
payroll_account             2
emc_account                 2
debit_card                  2
em_account_p                2
em_acount                   2
dtype: int64

In [16]:
df_product.isnull().sum()

Unnamed: 0             0
pk_cid                 0
pk_partition           0
short_term_deposit     0
loans                  0
mortgage               0
funds                  0
securities             0
long_term_deposit      0
em_account_pp          0
credit_card            0
payroll               61
pension_plan          61
payroll_account        0
emc_account            0
debit_card             0
em_account_p           0
em_acount              0
dtype: int64

In [12]:
df_sociodemo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   pk_cid        int64  
 2   pk_partition  object 
 3   country_id    object 
 4   region_code   float64
 5   gender        object 
 6   age           int64  
 7   deceased      object 
 8   salary        float64
dtypes: float64(2), int64(3), object(4)
memory usage: 409.4+ MB


In [17]:
df_sociodemo.nunique()

Unnamed: 0      5962924
pk_cid           456373
pk_partition         17
country_id           41
region_code          52
gender                2
age                 104
deceased              2
salary           258629
dtype: int64

In [18]:
df_sociodemo.isnull().sum()

Unnamed: 0            0
pk_cid                0
pk_partition          0
country_id            0
region_code        2264
gender               25
age                   0
deceased              0
salary          1512103
dtype: int64

In [13]:
df_comercial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   pk_cid           int64  
 2   pk_partition     object 
 3   entry_date       object 
 4   entry_channel    object 
 5   active_customer  float64
 6   segment          object 
dtypes: float64(1), int64(2), object(4)
memory usage: 318.5+ MB


In [19]:
df_comercial.nunique()

Unnamed: 0         5962924
pk_cid              456373
pk_partition            17
entry_date            1499
entry_channel           68
active_customer          2
segment                  3
dtype: int64

In [20]:
df_comercial.isnull().sum()

Unnamed: 0              0
pk_cid                  0
pk_partition            0
entry_date              0
entry_channel      133033
active_customer         0
segment            133944
dtype: int64

In [21]:
df_comercial.value_counts(normalize=True)*100

Unnamed: 0  pk_cid   pk_partition  entry_date  entry_channel  active_customer  segment           
0           1375586  2018-01-28    2018-01-12  KHL            1.0              02 - PARTICULARES     0.000017
9368210     1361521  2019-01-28    2017-11-26  KHF            1.0              03 - UNIVERSITARIO    0.000017
9368186     1361433  2019-01-28    2017-11-26  KHE            1.0              03 - UNIVERSITARIO    0.000017
9368185     1361432  2019-01-28    2017-11-26  KHE            0.0              03 - UNIVERSITARIO    0.000017
9368184     1361431  2019-01-28    2017-11-26  KHK            0.0              02 - PARTICULARES     0.000017
                                                                                                       ...   
5060674     1359055  2018-08-28    2017-11-21  KHE            0.0              03 - UNIVERSITARIO    0.000017
5060673     1359056  2018-08-28    2017-11-21  KHE            0.0              03 - UNIVERSITARIO    0.000017
5060672     1359165  2